In [11]:
import re
import keras.models
import spacy
import fitz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

nlp = spacy.load('model/model_job')

KeyboardInterrupt: 

In [ ]:
def extract_text(filePath, remove_line=False):
    with fitz.open(filePath) as doc:
        finaltext = ""
        for page in doc:
            text = page.get_text()
            text = text.replace("\n"," ")
            text = text.replace("[^a-zA-Z0-9]", " ");
            text = re.sub('\W+',' ', text)
            text = re.sub('[^A-Za-z0-9]',' ', text)
            finaltext += text

        if remove_line:
            finaltext = text = re.sub('\s', " ", text)

    return finaltext

def extract_job(text, nlp):
    doc = nlp(text)

    job = set()

    for a in doc.ents:
        job.add(str(a).lower())

    return job

def seq_and_pad(sentences, tokenizer):
    sequences = tokenizer.texts_to_sequences(sentences)

    padded_sequences = pad_sequences(sequences, maxlen=30, padding='post')


    return padded_sequences

In [ ]:
df = pd.read_csv('sample/train.csv')

df.head()

In [ ]:
df = df.loc[:, 'Email':'hasil_rekomendasi']
df['key'] = df['key'].astype('str')
df

In [ ]:
df.columns

In [ ]:
traits = 'CC'

feature = []
label = []

for i, row in df.iterrows():
    feature.append(df.at[i, 'key'])
    label.append(df.at[i, traits])

feature_train = feature[:84]
feature_test = feature[84:]

label_train = label[:84]
label_test = label[84:]

tokenizer = Tokenizer(num_words=1000, oov_token='OOV')
tokenizer.fit_on_texts(feature)

feature_train = np.asarray(seq_and_pad(feature_train, tokenizer))
feature_test = np.asarray(seq_and_pad(feature_test, tokenizer))

label_train = np.asarray(label_train)
label_test = np.asarray(label_test)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000, 16, input_length=30),
    tf.keras.layers.Conv1D(256, 3, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Set the training parameters
model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer='adam')

# Print the model summary
history = model.fit(feature_train,
                    label_train,
                    epochs=100,
                    validation_data=(feature_test, label_test),
                    verbose=2)

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, f'val_{metric}'])
    plt.show()

plot_graphs(history, "loss")

In [ ]:
model.evaluate(feature_test, label_test)

In [ ]:
pred = model.predict(feature_test)
result = datas = {'pred':pred, 'true':label_test}
for i in range(len(label_test)):
    print('pred : {} label : {}'.format(round(pred[i][0]), label[i]))

In [ ]:
model.save('{}.h5'.format(traits))